# Data Science I - Projeto final

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

In [ ]:
df = pd.read_csv('titanic-data-6.csv')

## Perguntas (Questions)

    1. Quais são os números de sobreviventes?
    2. Em qual classe havia mais familiares?
    3. Qual é o número de crianças a bordo?
    4. Qual é a proporção de crianças por gênero?
    5. Qual é a proporção de crianças por classe?
    6. Qual é a proporção de crianças que sobreviveram?
    

## Preparar dados (Wrangle data)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['Sex'].nunique()

In [ ]:
# Checking if exists null fields
df.isnull().any()

In [ ]:
# Filling null ages with mean
df['Age'].fillna(round(df['Age'].mean()), inplace=True)

In [ ]:
# Cheking if are there passengers without on board place and that weren't survived.
df[df['Survived'] == 0]['Embarked'].isnull().any().sum()

In [ ]:
# Removing passengers that weren't on board or without boarding place. No one of them were checked as not survived.
df['Embarked'].dropna(how='any', inplace=True)

In [ ]:
# Removing the 'Ticket' column, it doesn't look be an essencial info
df.drop(labels='Ticket', axis=1, inplace=True)

In [ ]:
# Changing 'Embarked' column abbreviated values to full values
embarked_places = {'S': 'Southampton', 'C': 'Cherbourg', 'Q': 'Queenstown'}
df['Embarked'].replace(embarked_places.keys(), embarked_places.values(), inplace=True)

In [ ]:
survived = {'0':'No', '1':'Yes'}
df['Survived'] = df['Survived'].astype(str).replace(survived.keys(),survived.values())

In [ ]:
def age_category(age):
    # Used as reference:
    # https://www.statcan.gc.ca/eng/concepts/definitions/age2
    if(age < 15):
        return 'Children'
    elif(age < 25):
        return 'Youth'
    elif(age < 65):
        return 'Adult'
    else:
        return 'Senior'
    
df['AgeCategory'] = df['Age'].apply(age_category)

In [ ]:
df.head()

## Explorar (Explore)

In [ ]:
df.head()

In [ ]:
pd.plotting.scatter_matrix(df[['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']], figsize=(15,15));

In [ ]:
df.hist(figsize=(15,15));

In [ ]:
plt.close('all')
sns.set_style('whitegrid')

### 1. Quais são os números de sobreviventes? Geral, por classe, gênero e taxa

#### Sobreviventes

In [ ]:


title = 'TITANIC SURVIVORS\nTotal passengers sample: {count}'.format(count=df.shape[0])

fig, ax = plt.subplots(1, 2)
fig.suptitle(title, fontsize='x-large', horizontalalignment='center', verticalalignment='bottom')

# Redismensionar o layout automaticamente
plt.tight_layout()
# Ajuste de posicionamento em relação ao topo
plt.subplots_adjust(top=.9)
# Definição da largura dos gráficos
fig.set_figwidth(14)

# Definição do dataframe de proporções
df_proportions = df['Survived'].value_counts(normalize=True).reset_index().rename(columns={'index':'Survived','Survived':'Proportions'})
df_proportions['Proportions'] = df_proportions['Proportions'] * 100

# Definição dos títulos
ax[0].set_title('Survivors count')
ax[1].set_title('Survivors proportions')

# Gerar os gráficos com seaborn
sns.countplot(data=df, x='Survived', ax=ax[0], palette='Blues_d')
sns.barplot(data=df_proportions, x='Survived', y='Proportions', ax=ax[1], palette='Blues_d');


In [ ]:
df_proportions

#### Sobreviventes por gênero e classe

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Survived', hue='Sex', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

In [ ]:
df_filtered = df.groupby(['Pclass', 'Sex', 'Survived']).count()['Age'].reset_index()
df_filtered.rename(columns={'Age':'Count'}, inplace=True)

df_filtered

#### Sobrevivents por local de embarque

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Survived', hue='Sex', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

#### Local de embarque por classe e gênero

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Pclass', hue='Sex', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

#### Classificação de idade por classe, local de embarque e sobreviventes

In [ ]:
ax_by_class = sns.factorplot(data=df, x='AgeCategory', hue='Survived', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

In [ ]:
ax_by_class = sns.factorplot(data=df, x='AgeCategory', hue='Survived', col='Embarked', kind='count', palette='Blues_d')
ax_by_class.set_titles('{col_name}');

In [ ]:
sns.violinplot(data=df, x='Pclass', y='Age', col='Pclass', hue='Sex', split=True, inner='quart', palette='Blues_d');


In [ ]:
sns.violinplot(data=df, x='Sex',y='Pclass', hue='Sex', split=True, inner='quart', palette='Blues_d');


In [ ]:
ax_by_class = sns.factorplot(data=df, x='Sex', y='Parch', col='Pclass', kind='bar', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

#### Contagem de pessoas por gênero em cada classe

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Sex', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

In [ ]:
df_surv.shape

In [ ]:
df_dead.shape

In [ ]:
df[['SibSp','Parch']].value_counts()

In [ ]:
ax_by_class = sns.factorplot(data=df, x='SibSp', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

In [ ]:
ax_by_class = sns.factorplot(data=df, x='Parch', col='Pclass', kind='count', palette='Blues_d')
ax_by_class.set_titles('Class {col_name}');

## Perguntas
    1. Quais são os números de sobreviventes?
    2. Em qual classe havia mais familiares?
    3. Qual é o número de crianças a bordo?
    4. Qual é a proporção de crianças por gênero?
    5. Qual é a proporção de crianças por classe?
    6. Qual é a proporção de crianças que sobreviveram?
    

## Conclusões    
    1. Com base na amostra analisada, sobreviveram apenas 38,4% (342) do total de 891 pessoas a bordo, 61,6% (549) morreram.
    2. 